In [83]:
import numpy as np
from tensorflow.keras import layers, models
import tensorflow as tf

In [84]:
import pandas as pd
from src.utils.path_utils import get_raw_data_dir

# Build the file path for the processed data file.
path = str(get_raw_data_dir()) + "/S3M6F1O1_dataset.csv"

# Load the data file
data = pd.read_csv(path)

In [85]:
# Get X data and y data
y_data = data['label'].values
X_data = data.drop(columns=['label']).values

# Transform to numpy array
X_data = np.array(X_data)
y_data = np.array(y_data)

# Determine the number of unique classes in the labels.
num_unique_labels = data['label'].nunique()

In [86]:
X_data.shape[1]

1000

In [87]:
num_unique_labels

3

In [88]:
LSTM_model = models.Sequential([
    layers.Input(shape=(X_data.shape[1],)),  # Accept 2D input (features,)
    layers.Reshape((1, -1)),  # Automatically add a time‐step dimension
    layers.Dense(32, activation='tanh'),
    layers.LSTM(64, unroll=True),
    layers.Dense(32, activation='tanh'),
    layers.Dense(num_unique_labels, activation='softmax')
])

In [89]:
LSTM_model.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [90]:
LSTM_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_2 (Reshape)             │ (None, 1, 1000)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1, 32)          │        32,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,043 (230.64 KB)

 Trainable params: 59,043 (230.64 KB)

 Non-trainable params: 0 (0.00 B)

In [91]:
from sklearn.model_selection import train_test_split

# Split the data into 80% training and 20% validation sets.
X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
)

In [92]:
X_train.shape

(873, 1000)

In [93]:
LSTM_model.fit(X_train, y_train,
       validation_data=(X_val, y_val),
       epochs=5,
       batch_size=32)

Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4285 - loss: 1.0349 - val_accuracy: 0.4703 - val_loss: 1.0297
Epoch 2/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4446 - loss: 1.0149 - val_accuracy: 0.4703 - val_loss: 1.0292
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4733 - loss: 1.0256 - val_accuracy: 0.3562 - val_loss: 1.0525
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4254 - loss: 1.0159 - val_accuracy: 0.4703 - val_loss: 1.0303
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4627 - loss: 1.0331 - val_accuracy: 0.4703 - val_loss: 1.0362
